# MNLP Homework 2 - OCRed text cleaning

In [ ]:
import matplotlib.pyplot as plt
import transformers
import pandas as pd
import numpy as np
import torch
import json
import sys
import os

from dotenv import load_dotenv
from huggingface_hub import login

# import torch
# import datasets
# import huggingface_hub

In [ ]:
cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

print(os.getcwd())

if '.env' in os.listdir(os.getcwd()):
    load_dotenv()
    path = os.environ['DATA_PATH']
    login(token=os.environ['HF_TOKEN'])    
else:
    pass
    #path = "C:\Users\FEDERICO\Desktop\MNLP-Homework2\data\ita"
    


In [ ]:
datasetRaw = {}

for i in [ocred_fn, cleaned_fn]:
    if i not in os.listdir(path):
        print(f"ERROR 404 ! File {i} not Found...")

    file_path = os.path.join(path, i)
    with open(file_path, 'r') as f:
        datasetRaw[i.split('.')[0]] = json.load(f)
        f.close()

In [ ]:
print(datasetRaw['original_ocr'].keys())
print(len(datasetRaw['original_ocr'].keys()))

In [ ]:
# print(datasetRaw['original_ocr']['1'])
#print(datasetRaw['cleaned']['1'])

# Models import


### Minerva

In [ ]:
minervaId = "sapienzanlp/Minerva-1B-base-v1.0"

minervaPipeline = transformers.pipeline(
    "text-generation",
    model=minervaId,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    trust_remote_code=True
)